In [14]:
##########################################Diffrent model training codes############################

import os, glob, random
import numpy as np
from tqdm import tqdm
from pathlib import Path
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# ----------------------------
# 1. CONFIG & HYPERPARAMETERS
# ----------------------------
DATA_DIR    = "roi_dataset"
SEQ_LEN     = 25
IMG_SIZE    = 224
BATCH_SIZE  = 8
DEVICE      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS      = 30
PATIENCE    = 5
LR          = 1e-4
DROP_P      = 0.3  # dropout probability

# ----------------------------
# 2. BUILD (frames_folder, label) LIST
# ----------------------------
classes = sorted([d.name for d in Path(DATA_DIR).iterdir() if d.is_dir()])
class2idx = {c:i for i,c in enumerate(classes)}
print(class2idx)
NUM_CLASSES = len(classes)

all_samples = []
for cls in classes:
    for vid_folder in (Path(DATA_DIR)/cls).iterdir():
        if vid_folder.is_dir():
            all_samples.append((str(vid_folder), class2idx[cls]))

# Stratified split: train 80%, val 10%, test 10%
labels = [lbl for _, lbl in all_samples]
train_val, test = train_test_split(all_samples, test_size=0.1,
                                   stratify=labels, random_state=42)
labels_tv = [lbl for _, lbl in train_val]
train, val    = train_test_split(train_val, test_size=0.1111,
                                 stratify=labels_tv, random_state=42)

# ----------------------------
# 3. DATASET & DATALOADER
# ----------------------------
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

class VideoFrameSequence(Dataset):
    def __init__(self, samples, seq_len, transform):
        self.samples  = samples
        self.seq_len  = seq_len
        self.transform= transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        folder, label = self.samples[idx]
        frames = sorted(glob.glob(os.path.join(folder, "*.jpg")),
                        key=lambda x: int(Path(x).stem.split('_')[-1]))
        if len(frames) < self.seq_len:
            frames += [frames[-1]] * (self.seq_len - len(frames))
        else:
            step = max(1, len(frames)//self.seq_len)
            frames = frames[::step][:self.seq_len]

        seq = []
        for f in frames:
            img = Image.open(f).convert("RGB")
            seq.append(self.transform(img))
        seq = torch.stack(seq)  # (T, C, H, W)
        return seq, label

train_ds = VideoFrameSequence(train, SEQ_LEN, transform)
val_ds   = VideoFrameSequence(val,   SEQ_LEN, transform)
test_ds  = VideoFrameSequence(test,  SEQ_LEN, transform)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# ----------------------------
# 4. CALLBACKS
# ----------------------------
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience, self.delta = patience, delta
        self.best_loss, self.counter = np.Inf, 0
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss + self.delta < self.best_loss:
            self.best_loss, self.counter = val_loss, 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# ----------------------------
# 5. MODEL DEFINITIONS
# ----------------------------
def get_backbone():
    m = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    m.fc = nn.Identity()
    return m

class CNN_RNN(nn.Module):
    def __init__(self, backbone, rnn_type="LSTM", bidir=False, drop_p=DROP_P):
        super().__init__()
        self.backbone = backbone
        hidden = 256
        rnn_cls = {"LSTM": nn.LSTM, "GRU": nn.GRU}[rnn_type]
        self.rnn = rnn_cls(input_size=512, hidden_size=hidden,
                           batch_first=True, bidirectional=bidir)
        mult = 2 if bidir else 1
        self.dropout = nn.Dropout(drop_p)
        self.head = nn.Linear(hidden*mult, NUM_CLASSES)

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B*T, C, H, W)
        feats = self.backbone(x)
        feats = feats.view(B, T, -1)
        out, _ = self.rnn(feats)
        x_last = out[:, -1, :]
        x_drop = self.dropout(x_last)
        return self.head(x_drop)

class ResNetClassifier(nn.Module):
    def __init__(self, drop_p=DROP_P):
        super().__init__()
        m = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        in_feats = m.fc.in_features
        m.fc = nn.Sequential(
            nn.Dropout(drop_p),
            nn.Linear(in_feats, NUM_CLASSES)
        )
        self.model = m

    def forward(self, x):
        x = x.mean(dim=1)
        return self.model(x)

class ViTClassifier(nn.Module):
    def __init__(self, drop_p=DROP_P):
        super().__init__()
        m = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_V1)
        hidden = m.hidden_dim
        m.heads = nn.Sequential(
            nn.Dropout(drop_p),
            nn.Linear(hidden, NUM_CLASSES)
        )
        self.model = m

    def forward(self, x):
        x = x.mean(dim=1)
        return self.model(x)

class TempTransformer(nn.Module):
    def __init__(self, num_layers=2, nhead=8, drop_p=DROP_P):
        super().__init__()
        self.backbone = get_backbone()
        self.pos_emb = nn.Parameter(torch.randn(SEQ_LEN, 512))
        encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=nhead)
        self.trans_enc = nn.TransformerEncoder(encoder_layer, num_layers)
        self.dropout = nn.Dropout(drop_p)
        self.head = nn.Linear(512, NUM_CLASSES)

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.view(B*T, C, H, W)
        feats = self.backbone(x).view(B, T, -1)
        feats = feats + self.pos_emb.unsqueeze(0)
        out = self.trans_enc(feats.permute(1,0,2))
        x_last = out[-1]
        x_drop = self.dropout(x_last)
        return self.head(x_drop)

# instantiate all models
models_dict = {
    "CNN-LSTM":    CNN_RNN(get_backbone(), "LSTM",  False),
    "CNN-GRU":     CNN_RNN(get_backbone(), "GRU",   False),
    "CNN-BiLSTM":  CNN_RNN(get_backbone(), "LSTM",  True),
    "ResNet":      ResNetClassifier(),
    "ViT":         ViTClassifier(),
    "Transformer": TempTransformer()
}

# ----------------------------
# 6. TRAIN & VALIDATE
# ----------------------------
def train_validate(model, name):
    model.to(DEVICE)
    opt   = optim.Adam(model.parameters(), lr=LR)
    sched = optim.lr_scheduler.ReduceLROnPlateau(opt, patience=2, factor=0.5)
    es    = EarlyStopping(patience=PATIENCE)

    history = {"train_loss":[], "val_loss":[], "train_acc":[], "val_acc":[]}
    best_val = np.Inf

    for epoch in range(1, EPOCHS+1):
        # Train
        model.train()
        total, correct, run_loss = 0,0,0
        for x,y in tqdm(train_loader, desc=f"{name} Epoch {epoch} [Train]"):
            x,y = x.to(DEVICE), y.to(DEVICE)
            opt.zero_grad()
            logits = model(x)
            loss   = nn.CrossEntropyLoss()(logits, y)
            loss.backward(); opt.step()
            preds = logits.argmax(1)
            total += y.size(0); correct += (preds==y).sum().item()
            run_loss += loss.item()*y.size(0)
        train_loss = run_loss/total; train_acc = correct/total

        # Validate
        model.eval()
        total, correct, run_loss = 0,0,0
        for x,y in tqdm(val_loader, desc=f"{name} Epoch {epoch} [Val]"):
            x,y = x.to(DEVICE), y.to(DEVICE)
            with torch.no_grad():
                logits = model(x)
                loss   = nn.CrossEntropyLoss()(logits, y)
            preds = logits.argmax(1)
            total += y.size(0); correct += (preds==y).sum().item()
            run_loss += loss.item()*y.size(0)
        val_loss = run_loss/total; val_acc = correct/total

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        print(f"{name} Epoch {epoch}/{EPOCHS} "
              f"Train {train_loss:.4f}/{train_acc:.3f} "
              f"Val   {val_loss:.4f}/{val_acc:.3f}")

        sched.step(val_loss)
        es(val_loss)
        if val_loss < best_val:
            best_val = val_loss
            torch.save(model.state_dict(), f"{name}_best.pth")
        if es.early_stop:
            print("→ Early stopping")
            break

    # Plot & save curves
    epochs = range(1, len(history["train_loss"])+1)
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(epochs, history["train_loss"], label="train")
    plt.plot(epochs, history["val_loss"],   label="val")
    plt.title(f"{name} Loss"); plt.legend()
    plt.subplot(1,2,2)
    plt.plot(epochs, history["train_acc"], label="train")
    plt.plot(epochs, history["val_acc"],   label="val")
    plt.title(f"{name} Accuracy"); plt.legend()
    plt.tight_layout()
    plt.savefig(f"{name}_curves.png")
    plt.close()

    # Load best, evaluate on test
    model.load_state_dict(torch.load(f"{name}_best.pth"))
    y_true, y_pred = [], []
    model.eval()
    with torch.no_grad():
        for x,y in test_loader:
            x = x.to(DEVICE)
            logits = model(x)
            preds = logits.argmax(1).cpu().tolist()
            y_pred += preds
            y_true+= y.tolist()

    report = classification_report(y_true, y_pred, target_names=classes)
    cm = confusion_matrix(y_true, y_pred)

    # save report
    with open(f"{name}_report.txt", 'w') as f:
        f.write(report)

    # plot & save cm
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=classes, yticklabels=classes)
    plt.title(f"{name} Confusion Matrix")
    plt.ylabel("True")
    plt.xlabel("Pred")
    plt.savefig(f"{name}_confusion_matrix.png")
    plt.close()

    test_acc = np.mean(np.array(y_pred) == np.array(y_true))
    return test_acc

# ----------------------------
# 7. RUN ALL MODELS & SELECT BEST
# ----------------------------
results = {}
for name, model in models_dict.items():
    acc = train_validate(model, name)
    results[name] = acc

best_model = max(results, key=results.get)
print(f"Best model: {best_model} with test accuracy {results[best_model]:.3f}")


{'cover': 0, 'defense': 1, 'flick': 2, 'hook': 3, 'late_cut': 4, 'lofted': 5, 'pull': 6, 'square_cut': 7, 'straight': 8, 'sweep': 9}


/home2/rahul.ku/myenv/lib/python3.12/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
CNN-LSTM Epoch 1 [Val]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:09<00:00,  2.64it/s]


CNN-LSTM Epoch 1/30 Train 1.7701/0.408 Val   1.3818/0.481


CNN-LSTM Epoch 2 [Val]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:06<00:00,  3.43it/s]


CNN-LSTM Epoch 2/30 Train 1.0064/0.679 Val   0.9564/0.693


...

Transformer Epoch 10/30 Train 0.0018/1.000 Val   1.2287/0.714


Transformer Epoch 11 [Val]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:05<00:00,  4.30it/s]


Transformer Epoch 11/30 Train 0.0018/1.000 Val   1.2384/0.709


Transformer Epoch 12 [Val]: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:05<00:00,  4.24it/s]


Transformer Epoch 12/30 Train 0.0013/1.000 Val   1.2803/0.704
→ Early stopping
Best model: CNN-BiLSTM with test accuracy 0.757
